<a href="https://cognitiveclass.ai"><img src = "https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/Logos/organization_logo/organization_logo.png" width = 400> </a>

<h1 align=center><font size = 5>Peer Review Final Assignment</font></h1>

## Introduction


In this lab, you will build an image classifier using the VGG16 pre-trained model, and you will evaluate it and compare its performance to the model we built in the last module using the ResNet50 pre-trained model. Good luck!

## Table of Contents

<div class="alert alert-block alert-info" style="margin-top: 20px">

<font size = 3>    

1. <a href="#item41">Download Data 
2. <a href="#item42">Part 1</a>
3. <a href="#item43">Part 2</a>  
4. <a href="#item44">Part 3</a>  

</font>
    
</div>

<a id="item41"></a>

## Download Data

Use the <code>wget</code> command to download the data for this assignment from here: https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0321EN/data/concrete_data_week4.zip

Use the following cells to download the data.

In [ ]:
# !wget https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0321EN/data/concrete_data_week4.zip

In [ ]:
# !unzip -q concrete_data_week4.zip

After you unzip the data, you fill find the data has already been divided into a train, validation, and test sets.

<a id="item42"></a>

## Part 1

In this part, you will design a classifier using the VGG16 pre-trained model. Just like the ResNet50 model, you can import the model <code>VGG16</code> from <code>keras.applications</code>.

You will essentially build your classifier as follows:
1. Import libraries, modules, and packages you will need. Make sure to import the *preprocess_input* function from <code>keras.applications.vgg16</code>.
2. Use a batch size of 100 images for both training and validation.
3. Construct an ImageDataGenerator for the training set and another one for the validation set. VGG16 was originally trained on 224 × 224 images, so make sure to address that when defining the ImageDataGenerator instances.
4. Create a sequential model using Keras. Add VGG16 model to it and dense layer.
5. Compile the mode using the adam optimizer and the categorical_crossentropy loss function.
6. Fit the model on the augmented data using the ImageDataGenerators.

Use the following cells to create your classifier.

In [2]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications.vgg16 import preprocess_input
import os
import numpy as np
import matplotlib.pyplot as plt

2025-03-08 12:08:42.208009: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1741415922.228307   51133 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1741415922.234064   51133 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-03-08 12:08:42.254391: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
train_batch_size, test_batch_size = 100, 100
image_resize = 224

In [8]:
data_generator = ImageDataGenerator(preprocessing_function=preprocess_input)

In [9]:
train_generator = data_generator.flow_from_directory(
    'concrete_data_week4/train',
    target_size=(image_resize, image_resize),
    batch_size=train_batch_size,
    class_mode='categorical')

Found 30001 images belonging to 2 classes.


In [10]:
valid_generator = data_generator.flow_from_directory(
    'concrete_data_week4/valid',
    target_size=(image_resize, image_resize),
    batch_size=train_batch_size,
    class_mode='categorical')

Found 9501 images belonging to 2 classes.


In [11]:
test_generator = data_generator.flow_from_directory(
    'concrete_data_week4/test',
    target_size=(image_resize, image_resize),
    batch_size=test_batch_size,
    class_mode='categorical', 
    shuffle=False)

Found 500 images belonging to 2 classes.


In [16]:
len(test_generator[0][0][:5])

5

In [17]:
test_generator[0][0][:5]

array([[[[119.061   , 105.221   , 100.32    ],
         [118.061   , 104.221   ,  99.32    ],
         [119.061   , 105.221   , 100.32    ],
         ...,
         [121.061   , 110.221   , 104.32    ],
         [120.061   , 109.221   , 103.32    ],
         [119.061   , 108.221   , 102.32    ]],

        [[118.061   , 104.221   ,  99.32    ],
         [118.061   , 104.221   ,  99.32    ],
         [119.061   , 105.221   , 100.32    ],
         ...,
         [120.061   , 109.221   , 103.32    ],
         [119.061   , 108.221   , 102.32    ],
         [118.061   , 107.221   , 101.32    ]],

        [[118.061   , 104.221   ,  99.32    ],
         [118.061   , 104.221   ,  99.32    ],
         [120.061   , 106.221   , 101.32    ],
         ...,
         [119.061   , 108.221   , 102.32    ],
         [118.061   , 107.221   , 101.32    ],
         [117.061   , 106.221   , 100.32    ]],

        ...,

        [[104.061   ,  93.221   ,  87.32    ],
         [102.061   ,  91.221   ,  85.32    ]

In [ ]:
model = Sequential()
model.add(VGG16(
    include_top=False,
    pooling='avg',
    weights='imagenet',
    ))
model.add(Dense(2, activation='softmax'))

In [ ]:
model.layers

In [ ]:
model.layers[0].layers

In [ ]:
# Freezing VGG16 layers, as we don't want to train them
model.layers[0].trainable = False

In [ ]:
model.summary()

In [ ]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
num_epochs = 2
steps_per_epoch_training = len(train_generator)
steps_per_epoch_validation = len(valid_generator)

In [ ]:
history = model.fit(
    train_generator,
    steps_per_epoch=steps_per_epoch_training,
    epochs=num_epochs,
    validation_data=valid_generator,
    validation_steps=steps_per_epoch_validation,
    verbose=1
)

## Part 2

In this part, you will evaluate your deep learning models on a test data. For this part, you will need to do the following:

1. Load your saved model that was built using the ResNet50 model. 
2. Construct an ImageDataGenerator for the test set. For this ImageDataGenerator instance, you only need to pass the directory of the test images, target size, and the **shuffle** parameter and set it to False.
3. Use the **evaluate_generator** method to evaluate your models on the test data, by passing the above ImageDataGenerator as an argument. You can learn more about **evaluate_generator** [here](https://keras.io/models/sequential/).
4. Print the performance of the classifier using the VGG16 pre-trained model.
5. Print the performance of the classifier using the ResNet pre-trained model.


Use the following cells to evaluate your models.

In [ ]:
test_generator = data_generator.flow_from_directory(
    'concrete_data_week4/test',
    target_size=(image_resize, image_resize),
    batch_size=test_batch_size,
    class_mode='categorical',
    shuffle=False)

In [ ]:
resnet_model_path = 'classifier_resnet_model.h5'
resnet_model = tensorflow.keras.models.load_model(resnet_model_path)

In [ ]:
# Evaluating the model vgg16 on the test data
loss, accuracy = model.evaluate(test_generator, steps=len(test_generator), verbose=1)
print("Test accuracy using VGG16 model: ", accuracy)

In [ ]:
# Evaluating the model vgg16 on the test data
loss, accuracy = resnet_model.evaluate(test_generator, steps=len(test_generator), verbose=1)
print("Test accuracy using ResNet model: ", accuracy)

<a id="item44"></a>

## Part 3

In this model, you will predict whether the images in the test data are images of cracked concrete or not. You will do the following:

1. Use the **predict_generator** method to predict the class of the images in the test data, by passing the test data ImageDataGenerator instance defined in the previous part as an argument. You can learn more about the **predict_generator** method [here](https://keras.io/models/sequential/).
2. Report the class predictions of the first five images in the test set. You should print something list this:


    <ul style="list-style-type:none">
        <li>Positive</li>  
        <li>Negative</li> 
        <li>Positive</li>
        <li>Positive</li>
        <li>Negative</li>
    </ul>


Use the following cells to make your predictions.

In [ ]:
test_generator[0][0][:5]

In [ ]:
model.predict(test_generator[0][0][:5])

In [ ]:
resent_model.predict(test_generator[0][0][:5])

### Thank you for completing this lab!

This notebook was created by Alex Aklson.

This notebook is part of a course on **Coursera** called *AI Capstone Project with Deep Learning*. If you accessed this notebook outside the course, you can take this course online by clicking [here](https://cocl.us/DL0321EN_Coursera_Week4_LAB1).

<hr>

Copyright &copy; 2020 [IBM Developer Skills Network](https://cognitiveclass.ai/?utm_source=bducopyrightlink&utm_medium=dswb&utm_campaign=bdu). This notebook and its source code are released under the terms of the [MIT License](https://bigdatauniversity.com/mit-license/).